In [1]:
from pyscf import gto, lib
import numpy as np

np.set_printoptions(16, threshold=100000000, suppress=False, linewidth=300)

In [2]:
basis = {
    "C": "cc-pVTZ",
    "H": "cc-pVTZ",
    "Sb": "cc-pVTZ-PP",
}

basis_ecp = {
    "Sb": "cc-pVTZ-PP",
}

mol = gto.Mole(atom="""
Sb        -1.33937843      0.44597852     -1.27279684
Sb         1.33937843     -0.44597852     -1.27279684
C         -1.40429524      1.10441871      0.83468205
C         -2.16210130     -1.56132398     -0.84717555
C          2.16210130      1.56132398     -0.84717555
C          1.40429524     -1.10441871      0.83468205
H         -0.69918639      1.91987631      1.00872018
H         -1.16111477      0.29030616      1.51873028
H         -2.40124532      1.47235562      1.08516843
H         -2.02002046     -2.22909286     -1.69887295
H         -1.69052287     -2.01612927      0.02577778
H         -3.23450854     -1.49489801     -0.65423339
H          2.02002046      2.22909286     -1.69887295
H          3.23450854      1.49489801     -0.65423339
H          1.69052287      2.01612927      0.02577778
H          0.69918639     -1.91987631      1.00872018
H          2.40124532     -1.47235562      1.08516843
H          1.16111477     -0.29030616      1.51873028
""", basis=basis, ecp=basis_ecp).build()

## cint_prop

In [3]:
mol.has_ecp()

True

In [4]:
mol.nbas

130

In [5]:
mol.atom_charge(0)

23

In [6]:
mol.nao_nr()

366

In [7]:
mol.nao_cart()

410

In [8]:
mol.nao_2c()

732

## 2-center intor

### 2c-sph

In [9]:
# no-deriv
out = mol.intor("int2c2e")
out_c = out.transpose(1, 0)
assert out_c.flags.c_contiguous
lib.fp(out_c), out_c.shape[::-1]

(-935.2694687735325, (366, 366))

In [10]:
# deriv
out = mol.intor("int1e_igovlp")
out_c = out.transpose(0, 2, 1)
assert out_c.flags.c_contiguous
lib.fp(out_c), out_c.shape[::-1]

(-106.16961857524265, (366, 366, 3))

In [11]:
# no-deriv | shl
out = mol.intor("int1e_kin", shls_slice=[0, 12, 8, 18])
out_c = out.transpose(1, 0)
assert out_c.flags.c_contiguous
lib.fp(out_c), out_c.shape[::-1]

(-0.35004852378214735, (41, 34))

In [12]:
# deriv | shl
out = mol.intor("int1e_ipnuc", shls_slice=[0, 12, 8, 18])
out_c = out.transpose(0, 2, 1)
assert out_c.flags.c_contiguous
lib.fp(out_c), out_c.shape[::-1]

(45.968244650324706, (41, 34, 3))

In [13]:
# no-deriv
out = mol.intor("int2c2e")
out_c = lib.pack_tril(np.array(out.swapaxes(0, 1), order="C"))
out_c = out_c.transpose(0)
lib.fp(out_c), out_c.shape[::-1]

(1809.6733664015762, (67161,))

In [14]:
# deriv
out = mol.intor("int1e_igovlp")
out_c = lib.pack_tril(np.array(out.swapaxes(1, 2), order="C"))
out_c = out_c.transpose(0, 1)
lib.fp(out_c), out_c.shape[::-1]

(-40.08094621810594, (67161, 3))

In [15]:
# no-deriv | shl
out = mol.intor("int1e_kin", shls_slice=[8, 18, 8, 18])
out_c = lib.pack_tril(np.array(out.swapaxes(0, 1), order="C"))
out_c = out_c.transpose(0)
lib.fp(out_c), out_c.shape[::-1]

(0.5049496096560406, (595,))

In [16]:
# deriv | shl
out = mol.intor("int1e_ipnuc", shls_slice=[8, 18, 8, 18])
out_c = lib.pack_tril(np.array(out.swapaxes(1, 2), order="C"))
out_c = out_c.transpose(0, 1)
lib.fp(out_c), out_c.shape[::-1]

(4.499859812123924, (595, 3))

## 2-center ECP

### sph

In [41]:
# no-deriv
out = mol.intor("ECPscalar")
out_c = out.transpose(1, 0)
assert out_c.flags.c_contiguous
lib.fp(out_c), out_c.shape[::-1]

(0.7534281412150909, (366, 366))

In [34]:
# deriv
out = mol.intor("ECPscalar_ignuc")
out_c = out.transpose(0, 2, 1)
assert out_c.flags.c_contiguous
lib.fp(out_c), out_c.shape[::-1]

(-1.3426139111770714, (366, 366, 3))

In [35]:
# no-deriv | shl
out = mol.intor("ECPscalar", shls_slice=[0, 12, 8, 18])
out_c = out.transpose(1, 0)
assert out_c.flags.c_contiguous
lib.fp(out_c), out_c.shape[::-1]

(1.4186154309495258, (41, 34))

In [42]:
# deriv | shl
out = mol.intor("ECPscalar_iprinv", shls_slice=[0, 12, 8, 18])
out_c = out.transpose(0, 2, 1)
assert out_c.flags.c_contiguous
lib.fp(out_c), out_c.shape[::-1]

(-11.738302192911995, (41, 34, 3))

In [37]:
# no-deriv
out = mol.intor("ECPscalar")
out_c = lib.pack_tril(np.array(out.swapaxes(0, 1), order="C"))
out_c = out_c.transpose(0)
lib.fp(out_c), out_c.shape[::-1]

(-1.8388155709304113, (67161,))

In [38]:
# deriv
out = mol.intor("ECPscalar_iprinvip")
out_c = lib.pack_tril(np.array(out.swapaxes(1, 2), order="C"))
out_c = out_c.transpose(0, 1)
lib.fp(out_c), out_c.shape[::-1]

(-68.89774165957373, (67161, 9))

In [39]:
# no-deriv | shl
out = mol.intor("ECPscalar", shls_slice=[8, 18, 8, 18])
out_c = lib.pack_tril(np.array(out.swapaxes(0, 1), order="C"))
out_c = out_c.transpose(0)
lib.fp(out_c), out_c.shape[::-1]

(15.651229742132026, (595,))

In [40]:
# deriv | shl
out = mol.intor("ECPscalar_iprinvip", shls_slice=[8, 18, 8, 18])
out_c = lib.pack_tril(np.array(out.swapaxes(1, 2), order="C"))
out_c = out_c.transpose(0, 1)
lib.fp(out_c), out_c.shape[::-1]

(0.5549200392488471, (595, 9))

### cart

In [44]:
# no-deriv
out = mol.intor("ECPscalar_cart")
out_c = out.transpose(1, 0)
assert out_c.flags.c_contiguous
lib.fp(out_c), out_c.shape[::-1]

(1.6418489688115985, (410, 410))

In [45]:
# deriv
out = mol.intor("ECPscalar_ignuc_cart")
out_c = out.transpose(0, 2, 1)
assert out_c.flags.c_contiguous
lib.fp(out_c), out_c.shape[::-1]

(-16.389862409392386, (410, 410, 3))

In [46]:
# no-deriv | shl
out = mol.intor("ECPscalar_cart", shls_slice=[0, 12, 8, 18])
out_c = out.transpose(1, 0)
assert out_c.flags.c_contiguous
lib.fp(out_c), out_c.shape[::-1]

(1.3845811428580408, (47, 39))

In [52]:
# no-deriv | shl
out = mol.intor("int1e_nuc_cart", shls_slice=[0, 12, 8, 18])
out_c = out.transpose(1, 0)
assert out_c.flags.c_contiguous
lib.fp(out_c), out_c.shape[::-1]

(32.8336331060587, (47, 39))

In [47]:
# deriv | shl
out = mol.intor("ECPscalar_iprinv_cart", shls_slice=[0, 12, 8, 18])
out_c = out.transpose(0, 2, 1)
assert out_c.flags.c_contiguous
lib.fp(out_c), out_c.shape[::-1]

(-7.983349456393638, (47, 39, 3))

In [48]:
# no-deriv
out = mol.intor("ECPscalar_cart")
out_c = lib.pack_tril(np.array(out.swapaxes(0, 1), order="C"))
out_c = out_c.transpose(0)
lib.fp(out_c), out_c.shape[::-1]

(-9.966466894809031, (84255,))

In [49]:
# deriv
out = mol.intor("ECPscalar_iprinvip_cart")
out_c = lib.pack_tril(np.array(out.swapaxes(1, 2), order="C"))
out_c = out_c.transpose(0, 1)
lib.fp(out_c), out_c.shape[::-1]

(132.938348145421, (84255, 9))

In [50]:
# no-deriv | shl
out = mol.intor("ECPscalar_cart", shls_slice=[8, 18, 8, 18])
out_c = lib.pack_tril(np.array(out.swapaxes(0, 1), order="C"))
out_c = out_c.transpose(0)
lib.fp(out_c), out_c.shape[::-1]

(-20.31359441160549, (780,))

In [51]:
# deriv | shl
out = mol.intor("ECPscalar_iprinvip_cart", shls_slice=[8, 18, 8, 18])
out_c = lib.pack_tril(np.array(out.swapaxes(1, 2), order="C"))
out_c = out_c.transpose(0, 1)
lib.fp(out_c), out_c.shape[::-1]

(1.9270442333836884, (780, 9))